# Installation et imports

### Installation des modules 

In [327]:
%pip install Pandas


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [328]:
%pip install Matplotlib 


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [329]:
%pip install Scikit-learn


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [330]:
%pip install Numpy


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Import et données

In [331]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib as plt
import math


In [332]:
data = pd.read_csv("../df.csv")

# Répartition des données

### Répartition des données en base d’apprentissage et de tests en utilisant les algorithmes holdout 

In [333]:
from sklearn.model_selection import train_test_split

In [334]:
def holdout(dataframe, colonne_predict, num_rep, train_s, test_s ):
        liste_data_train_test = []
        liste_data = []
        clean_data = dataframe.drop(colonne_predict, axis = 1)
        y = dataframe[colonne_predict]
        for i in range(num_rep):
                X_train, X_test, y_train, y_test = train_test_split(clean_data,y, train_size= train_s, test_size = test_s)
                liste_data_train_test.append([X_train, X_test, y_train, y_test])
        return liste_data_train_test

In [335]:
liste_data_holdout = []
data_test = data[['descr_grav', 'latitude', 'longitude']]
liste_data_holdout  = holdout(data_test,"descr_grav", 5, 0.8, 0.2)

In [336]:
from sklearn.model_selection import LeaveOneOut
from sklearn.neighbors import KNeighborsClassifier

In [337]:
def leaveoneout(dataframe, colonne_predict):
    liste_data = []
    data = dataframe.sample(n=7000) #on prend 10% du dataframe
    data = data.reset_index(drop=True)
    X = data.drop(colonne_predict, axis = 1) 
    y = data[colonne_predict]
    LOO = LeaveOneOut()
    knn = KNeighborsClassifier(n_neighbors=5)
    
    for train_index, test_index in (LOO.split(X)):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        liste_data.append(y_pred)
        
    return liste_data

In [338]:
liste_y_pred = leaveoneout(data_test, "descr_grav")
print(liste_y_pred)

[array([0]), array([2]), array([0]), array([1]), array([2]), array([0]), array([1]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([1]), array([1]), array([1]), array([0]), array([0]), array([1]), array([1]), array([0]), array([0]), array([0]), array([0]), array([0]), array([1]), array([0]), array([2]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([1]), array([0]), array([0]), array([1]), array([1]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([2]), array([1]), array([1]), array([0]), array([0]), array([2]), array([0]), array([0]), array([1]), array([0]), array([2]), array([0]), array([0]), array([1]), array([1]), array([0]), array([0]), array([0]), array([0]), array([1]), array([1]), array([0]), array([0]), array([1]), array([2]), array([0]), array([2]), array([0]), array([1]), array([1]), array([0]), array([1]), array([1]), array([0]), array([0]), array([1]), array([1]), array([1]), arr

### Classification avec KNN

In [339]:
#on va prendre en entrée une ligne du df
def KnnClassification(ligne,df,k):
    dictList = [None] * len(df)
    firstk = []
    for index, row in df.iterrows():
        dictList[index] = np.linalg.norm(ligne - row)
    
    sorted_values = sorted(enumerate(dictList), key=lambda x: x[1])[:k]
    for index,value in sorted_values:
        firstk.append([df.loc[index]])
    return firstk

In [340]:
knn = KNeighborsClassifier(n_neighbors=5)
liste_data_y_pred = []
for i in range(len(liste_data_holdout)):
    X_train = liste_data_holdout[i][0]
    y_train = liste_data_holdout[i][2]
    X_test = liste_data_holdout[i][1]
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    liste_data_y_pred.append(y_pred)
print(liste_data_y_pred)

[array([2, 1, 1, ..., 0, 0, 0]), array([0, 0, 0, ..., 1, 0, 1]), array([1, 0, 0, ..., 0, 0, 0]), array([0, 0, 0, ..., 1, 2, 0]), array([1, 0, 0, ..., 0, 0, 1])]


In [341]:
#from scratch knn

X_test = liste_data_holdout[0][1]
X_test = X_test.reset_index(drop=True)
print(len(X_test))
#for i in range(len(X_test)):
 #   liste = KnnClassification(X_test.iloc[i], X_test, 5)
liste = KnnClassification(X_test.loc[0], X_test, 5)
print(liste)

14575
[[latitude     48.75000
longitude    -2.98333
Name: 0, dtype: float64], [latitude     48.75000
longitude    -2.98333
Name: 1864, dtype: float64], [latitude     48.75000
longitude    -2.98333
Name: 5586, dtype: float64], [latitude     48.75000
longitude    -2.98333
Name: 8742, dtype: float64], [latitude     48.71670
longitude    -2.96667
Name: 9162, dtype: float64]]


In [342]:
data = data[['descr_grav','descr_cat_veh','age']]
data.head()


,descr_grav,descr_cat_veh,age
0,0,2,40
1,0,1,50
2,0,1,49
3,0,1,52
4,0,1,58


In [343]:
#data = pd.read_csv("../df.csv")
#print(data.loc[65])
print('__')
t = KnnClassification(data.loc[65],data,5)

__


In [344]:
t

[[descr_grav        0
  descr_cat_veh     1
  age              44
  Name: 65, dtype: int64],
 [descr_grav        0
  descr_cat_veh     1
  age              44
  Name: 84, dtype: int64],
 [descr_grav        0
  descr_cat_veh     1
  age              44
  Name: 185, dtype: int64],
 [descr_grav        0
  descr_cat_veh     1
  age              44
  Name: 745, dtype: int64],
 [descr_grav        0
  descr_cat_veh     1
  age              44
  Name: 796, dtype: int64]]

In [345]:
a = 'f*o'

for e in a:
    print(ord(e))


102
42
111
